<a href="https://colab.research.google.com/github/Femme-js/am/blob/main/aml1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import random
import csv
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

In [ ]:
pwd

'/content'

In [ ]:
pd. set_option('display.max_rows', None, 'display.max_columns', None)

In [ ]:
train = pd.read_csv('drive/MyDrive/data/dataset52a7b21/dataset/train.csv', escapechar = "\\", quoting = csv.QUOTE_NONE)

In [ ]:
train.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [ ]:
train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903024 entries, 0 to 2903023
Data columns (total 5 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   TITLE           2902953 non-null  object
 1   DESCRIPTION     2179360 non-null  object
 2   BULLET_POINTS   2736761 non-null  object
 3   BRAND           2846287 non-null  object
 4   BROWSE_NODE_ID  2903024 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 110.7+ MB


In [ ]:
pd.get_dummies(train.BROWSE_NODE_ID)

In [ ]:
needed_Class = []
for i in train['BROWSE_NODE_ID'].value_counts().to_frame().index:
  if train['BROWSE_NODE_ID'].value_counts().to_frame().loc[i]['BROWSE_NODE_ID'] >= 1000:
    needed_Class.append(i)

In [ ]:
train = train.loc[train['BROWSE_NODE_ID'].isin(needed_Class)]

In [ ]:
train['COMBINE'] = train['TITLE'].fillna('') + train['DESCRIPTION'].fillna('') + train['BULLET_POINTS'].fillna('') + train['BRAND'].fillna('')

In [ ]:
train = train.loc[train['COMBINE']!='']

In [ ]:
train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077189 entries, 0 to 2903023
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   TITLE           2077135 non-null  object
 1   DESCRIPTION     1529598 non-null  object
 2   BULLET_POINTS   1975149 non-null  object
 3   BRAND           2040933 non-null  object
 4   BROWSE_NODE_ID  2077189 non-null  int64 
 5   COMBINE         2077189 non-null  object
dtypes: int64(1), object(5)
memory usage: 110.9+ MB


In [ ]:
X_train, _, y_train, _ = train_test_split(train, 
                                                  train.BROWSE_NODE_ID.values, 
                                                  test_size=0.7, 
                                                  random_state=42, 
                                                  stratify=train.BROWSE_NODE_ID.values)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train.COMBINE.values, 
                                                  X_train.BROWSE_NODE_ID.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=X_train.BROWSE_NODE_ID.values)
X_train['data_type'] = ['not_set']*X_train.shape[0]

X_train.loc[X_train['COMBINE'].isin(x_train), 'data_type'] = 'train'
X_train.loc[X_train['COMBINE'].isin(x_val), 'data_type'] = 'val'

X_train.groupby(['BROWSE_NODE_ID', 'data_type']).count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


TITLE  DESCRIPTION  BULLET_POINTS  BRAND  COMBINE
BROWSE_NODE_ID data_type                                                   
0              train        716          448            518    594      716
               val          179          113            127    148      179
1              train        433          386            419    428      433
               val          112          111            111    112      112
4              train       8201         3347           2999   3975     8201
               val         2050          841            763    991     2050
5              train      16622        10701          16263  16616    16622
               val         4473         2856           4370   4469     4473
6              train        899          589            863    890      899
               val          228          147            220    227      228
11             train       5180         4124           5084   5166     5180
               val         1318         1066           1302   1315     1318
16             train        378          343            356    376      378
               val           96           88             94     95       96
19             train        342          290            340    342      342
               val           86           76             86     86       86
22             train       2028         1516           1951   2022     2029
               val          677          375            662    676      677
23             train        414          292            338    406      414
               val          104           60             79    102      104
35             train        860          706            784    831      860
               val          216          176            196    209      216
36             train        500          387            466    487      500
               val          125           98            119    123      125
37             train        329          294            319    326      329
               val          104           86             99    103      104
38             train        766          531            597    674      766
               val          194          141            158    175      194
40             train       3525         1054           3491   3522     3525
               val          910          244            897    910      910
53             train        443          315            357    386      443
               val          112           79             86     94      112
55             train       2757         1960           2107   2398     2757
               val          689          495            530    596      689
58             train        571          425            494    557      571
               val          143          103            119    140      143
66             train        337          279            320    329      337
               val           84           75             80     84       84
69             train       1086          720            846    949     1086
               val          273          177            218    240      273
74             train       2303         1874           2210   2290     2303
               val          584          490            554    578      584
75             train        350          293            305    327      350
               val           88           76             79     82       88
76             train        688          482            565    673      688
               val          173          116            138    170      173
81             train        601          523            568    599      601
               val          155          137            150    154      155
82             train       1365          984           1046   1188     1365
               val          344          254            262    304      344
84             train        442          308            352    403      44

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

print('moving next!')

encoded_data_train = tokenizer.batch_encode_plus(
    X_train[X_train.data_type=='train'].COMBINE.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=50, 
    return_tensors='pt'
)

print('moving next!')

encoded_data_val = tokenizer.batch_encode_plus(
    X_train[X_train.data_type=='val'].COMBINE.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=50, 
    return_tensors='pt'
)

print('moving next!')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(X_train[X_train.data_type=='train'].BROWSE_NODE_ID.values)

print('moving next!')

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(X_train[X_train.data_type=='val'].BROWSE_NODE_ID.values)

print('moving next!')

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

print('completed!')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


moving next!


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


moving next!
moving next!
moving next!
moving next!
completed!


In [ ]:
label_num = X_train[X_train.data_type=='train'].BROWSE_NODE_ID.max()

In [ ]:
label_num

15051

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=label_num+1,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 15

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f"drive/MyDrive/data/BERT_epoch_{epoch}_{loss.item()/len(batch)}.pt")
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')